# Machine Learning Challenge

Below are 2 data challenges that test for your ability to:
- Wrangle/clean data to make it usable by a model
- Figure out how to set up X's and y's for a use case, given a dataset
- Write code to robustly and reproducibly preprocess data
- Pick/design the right model, and tune hyperparameters to get the best performance

You can use any programming language, model, and package to solve these problems. Let us know of any assumptions you make in your process.

#### Deliverables:
- A link to a github repository that contains:
    - Clearly commented code that was written to solve these problems
    - Your trained models stored in a file (`.pkl`, `.h5`, `.tar` - whatever is appropriate). The models must have `predict(X)` functions. 
    - A readme file that contains:
        - Instructions to easily access/load the above
        - A writeup explaining any significant design decisions and your reasons for making them. 
        - If needed, a brief writeup explaining anything you are particularly proud of in your implementation that you might want us to focus on

#### How we'll assess your work:
- Accuracy/RMSE of your model when predicting on held-out data
- How well various edge cases are handled when testing on held-out data. For example, if the held-out data contains:
    - A new column that wasn't present in the dataset given to you
    - New value in a categorical field that wasn't seen in the dataset given to you
    - NA values
- Efficiency of the code. 
    - Is it easy to understand? 
    - Are the variable names descriptive? 
    - Are there any variables created that aren't used? 
    - Is redundant code replaced with function calls? 
    - Is vectorized implementation used instead of nested for loops? 
    - Are classes defined and objects created where applicable? 
    - Are packages used to perform tasks instead of implementing them from scratch?
    
**NOTE:** Your stored models, once loaded, should *just work* when fed with our held-out data (which looks similar to the data we've given you). We won't do any preprocessing before we feed it into the model's `predict(X)` function; `predict(X)` should handle the preprocessing. Pay particular attention to handling the edge cases we've talked about.

Feel free to ask questions to clarify things. Submit everything you tried, not just the things that worked. I encourage you to try and showcase your talents. The more you go above and beyond what's expected, the more impressed we'll be. **Bonus points if you fit Keras/Tensorflow/Pytorch/Caffe models** in addition to your Linear/Tree-based models.

## 0. Import dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing as scale
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

import xgboost
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import mean_squared_error, accuracy_score, average_precision_score, precision_score, f1_score,recall_score, roc_auc_score

C:\Users\Prakash14\Anaconda3\envs\carnd-term1\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Task 1
`predictive_maintenance_dataset.csv` is a file that contains parameters and settings (`operational_setting_1`, `operational_setting_2`, `sensor_measurement_1`, `sensor_measurement_2`, etc.) for many wind turbines. There is a column called `unit_number` which specifies which turbine it is, and one called `status`, in which a value of 1 means the turbine broke down that day, and 0 means it didn't. Your task is to create a model that, when fed with operational settings and sensor measurements (`unit_number` and `time_stamp` will *not* be fed in), outputs 1 if the turbine will break down within the next 40 days, and 0 if not.

**NOTE:** The model should output 1 if the turbine is anywhere between 40 and 0 days away from failure, not *only* 40 days from failure.

In [ ]:
## What the data that we'll feed into your model's predict(X) function will look like:
# Notice what the operational_setting_3 column looks like
df_X = pd.read_csv("predictive_maintenance_dataset.csv").drop(labels=['status', 'unit_number', 'time_stamp'], axis='columns')
df_X.head()

### Import data

In [3]:
df = pd.read_csv("predictive_maintenance_dataset.csv", parse_dates=['time_stamp']).sort_values(by = ['unit_number', 'time_stamp'], ascending = True)
df.head()

,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,operational_setting_3,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,...,sensor_measurement_12,sensor_measurement_13,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21
73382,2,2017-04-01 12:00:00,0,-0.0018,0.0006,High,518.67,641.89,1583.84,1391.28,...,522.33,2388.06,8137.72,8.3905,0.03,391.0,2388.0,100.0,38.94,23.4585
90923,2,2017-04-02 12:00:00,0,0.0043,-0.0003,High,518.67,641.82,1587.05,1393.13,...,522.70,2387.98,8131.09,8.4167,0.03,NaN,2388.0,100.0,39.06,23.4085
82527,2,2017-04-03 12:00:00,0,0.0018,0.0003,High,518.67,641.55,1588.32,1398.96,...,522.58,2387.99,8140.58,8.3802,0.03,391.0,2388.0,100.0,39.11,23.4250
96521,2,2017-04-04 12:00:00,0,0.0035,-0.0004,High,518.67,641.68,1584.15,1396.08,...,522.49,2387.93,8140.44,8.4018,0.03,391.0,2388.0,100.0,39.13,23.5027
73137,2,2017-04-05 12:00:00,0,0.0005,0.0004,High,518.67,641.73,1579.03,1402.52,...,522.27,2387.94,8136.67,8.3867,0.03,390.0,2388.0,100.0,39.18,23.4234


### Data exploration

Check for non-numerical values, categorical columns

In [4]:
df.select_dtypes(include=['object']).dtypes

operational_setting_3    object
dtype: object

Are there any null columns and how many null values? 

In [5]:
df.isnull().sum()

unit_number                 0
time_stamp                  0
status                      0
operational_setting_1    7141
operational_setting_2    7196
operational_setting_3    7227
sensor_measurement_1     7209
sensor_measurement_2     7198
sensor_measurement_3     7190
sensor_measurement_4     7335
sensor_measurement_5     7244
sensor_measurement_6     7444
sensor_measurement_7     7213
sensor_measurement_8     7276
sensor_measurement_9     7207
sensor_measurement_10    7191
sensor_measurement_11    7180
sensor_measurement_12    7227
sensor_measurement_13    7115
sensor_measurement_14    7068
sensor_measurement_15    7257
sensor_measurement_16    7059
sensor_measurement_17    7167
sensor_measurement_18    7265
sensor_measurement_19    7195
sensor_measurement_20    7018
sensor_measurement_21    7165
dtype: int64

That's a lot of empty values! 

### Categorical value

Considering replacement with mode so the most common occurence is used to fill the NaNs, however, if this is a very influential variable it might affect our model significantly. 

Another option is to just replace it with the neighbouring values.

In [6]:
df['operational_setting_3'].fillna(df['operational_setting_3'].mode()[0], inplace=True)

Converting to dummy variable and concatenating them at the end to quantify categorical variables

In [7]:
df = pd.concat([df.drop('operational_setting_3', axis=1), pd.get_dummies(df.operational_setting_3)], axis=1)

### Numerical values

Some 7000 values are missing out of 144000, that's about 5%, a significant number. This could be valuable information that otherwise may skew our data if not used or replaced with simply a zero.

There are several ways we can approach the missing numerical values. We could use the mean or median values for the entire data set, or narrow down to those values of the individual units. 

For simplicity, since it is a time series and the data is chronologically sorted by machine numbers we just replace the missing variable with that of the previous hour. Assuming just sensor failure, this gives us  Assuming that would give us the most similar and likely variable that would existed there, if not perhaps for a sensor failure. 

In [7]:
#df.fillna(df.mean(axis=0), axis=0, inplace=True)
df.fillna(method='ffill', inplace=True)

### Constructing Labels

We have an interesting case here: Multi-class classification for predicting failure within a given time window

Essentially, we're trying to figure out a problem where given a set of parameters what is the likelihood that a certain unit fails within a 40 day timespan. A very useful business case. 


We can frame the ML problem by identifying the timestep of failure of turbine and backfilling 40 timeperiods with failures too. 

This labelled data set converts the time series problem to a supervised learning problem. 

In [8]:
df.groupby(['status']).count()

,unit_number,time_stamp,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
status,,,,,,,,,,,,,,,,,,,,,
0,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570,...,143570,143570,143570,143570,143570,143570,143570,143570,143570,143570
1,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,633,633,633,633,633


Let's replace all the 0s with NaNs and then we work backwards

In [9]:
df['status_new'] = df['status'].replace(0, np.NaN) 

We fill backward up to 40days. Thankfully the data is frequent, continuous, and daily. 

In [10]:
df['status_new'] = df['status_new'].fillna(method='bfill', limit=40) 
df['status_new'] = df['status_new'].fillna('0') #fill the rest with zeros

In [11]:
df.groupby(['status_new']).count()

,unit_number,time_stamp,status,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
status_new,,,,,,,,,,,,,,,,,,,,,
1.0,25953,25953,25953,25953,25953,25953,25953,25953,25953,25953,...,25953,25953,25953,25953,25953,25953,25953,25953,25953,25953
0,118250,118250,118250,118250,118250,118250,118250,118250,118250,118250,...,118250,118250,118250,118250,118250,118250,118250,118250,118250,118250


Our failure cases have increased from 633 to 25953! That's a 40 time increase as expected. But the added benefit is that our dataset isn't so skewed anymore! 

In [12]:
df_X = df.drop(['unit_number', 'status', 'time_stamp','status_new'], axis = 1)
df_y = df['status_new']

In [13]:
df_X.head()

,operational_setting_1,operational_setting_2,sensor_measurement_1,sensor_measurement_2,sensor_measurement_3,sensor_measurement_4,sensor_measurement_5,sensor_measurement_6,sensor_measurement_7,sensor_measurement_8,...,sensor_measurement_14,sensor_measurement_15,sensor_measurement_16,sensor_measurement_17,sensor_measurement_18,sensor_measurement_19,sensor_measurement_20,sensor_measurement_21,High,Low
73382,-0.0018,0.0006,518.67,641.89,1583.84,1391.28,14.62,21.60,554.53,2388.01,...,8137.72,8.3905,0.03,391.0,2388.0,100.0,38.94,23.4585,1,0
90923,0.0043,-0.0003,518.67,641.82,1587.05,1393.13,14.62,21.61,554.77,2387.98,...,8131.09,8.4167,0.03,391.0,2388.0,100.0,39.06,23.4085,1,0
82527,0.0018,0.0003,518.67,641.55,1588.32,1398.96,14.62,21.60,555.14,2388.04,...,8140.58,8.3802,0.03,391.0,2388.0,100.0,39.11,23.4250,1,0
96521,0.0035,-0.0004,518.67,641.68,1584.15,1396.08,14.62,21.61,554.25,2387.98,...,8140.44,8.4018,0.03,391.0,2388.0,100.0,39.13,23.5027,1,0
73137,0.0005,0.0004,518.67,641.73,1579.03,1402.52,14.62,21.61,555.12,2388.03,...,8136.67,8.3867,0.03,390.0,2388.0,100.0,39.18,23.4234,1,0


In [14]:
df_y.tail()

121541    1
101889    1
17932     1
10097     1
103638    1
Name: status_new, dtype: object

### Modelling

We first split the data into training and testing sets. 

In [16]:
xtrain, xval, ytrain, yval = train_test_split(df_X, df_y, test_size = 0.3, random_state = 19 ) 

In [17]:
ytrain=ytrain.astype(int)
yval=yval.astype(int)

In [18]:
print( xtrain.shape, xval.shape, ytrain.shape, yval.shape)

(100942, 25) (43261, 25) (100942,) (43261,)


### Model Selection

We evaluate a few simple yet robust classification models and choose the best one for our primary working model in the main notebook

In [19]:
def metrics(ytruth, pred):
    """
    Function to evaluate models against models 
    """
    print('accuracy score: ', accuracy_score(ytruth, pred))
    #print('RMSE:', mean_squared_error(ytest,pred))
    print('Recall score: ', recall_score(ytruth,pred))
    
    #print('average_precision_score: ', average_precision_score(ytest,pred))
    print('Precision Score: ',precision_score(ytruth,pred))
    print('F1_score: ',f1_score(ytruth, pred))
    #print('roc_auc_score: ', roc_auc_score(ytest, pred))

In [20]:
def logisticRegression(xtrain,xval, ytrain, yval):
    LR = LogisticRegression()
    model = LR.fit(xtrain, ytrain)
    pred = model.predict(xval)
    metrics(yval,pred)

In [21]:
def randomForestClassifier(xtrain,xval,ytrain,yval,n_estimators=25,min_samples_split=25,max_depth=5,random_state=72):
    RF = RandomForestClassifier(n_estimators = 25, min_samples_split=25, max_depth =5, random_state=72)
    
    model = RF.fit(xtrain,ytrain)
    pred = RF.predict(xval)
    metrics(yval, pred)

In [23]:
def xgbClassifier(xtrain,xval,ytrain,yval, max_depth=9, n_estimators=50, learning_rate=0.05, objective='binary:logistic'):
    xgb = xgboost.XGBClassifier( max_depth=9, n_estimators=50, learning_rate=0.05, objective='binary:logistic')
    
    model = xgb.fit(xtrain,ytrain.values)
    pred = xgb.predict(xval)
    metrics(yval, pred)

In [26]:
logisticRegression(xtrain,xval, ytrain, yval)

accuracy score:  0.8113312221169182
Recall score:  0.030930470347648262
Precision Score:  0.2944038929440389
F1_score:  0.0559796437659033


In [27]:
randomForestClassifier(xtrain,xval,ytrain,yval, n_estimators=20,min_samples_split=2,max_depth=25,random_state=72)

accuracy score:  0.9109128314185987
Recall score:  0.5866564417177914
Precision Score:  0.8809980806142035
F1_score:  0.704311799907933


In [28]:
xgbClassifier(xtrain,xval,ytrain,yval)

43261
accuracy score:  0.9435750444973533
Recall score:  0.7998466257668712
Precision Score:  0.8773307163886163
F1_score:  0.8367988232934412


C:\Users\Prakash14\Anaconda3\envs\carnd-term1\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Although the random forest model has a relatively high accuracy of 91, the xgb has a higher accuracy and a much higher recall score of 0.81.

I think this value is of more importance as it highlights the number of times a machine required maintenance and it was correctly flagged to be repaired. 

In the real world, poor performance with this metric would lead to greater repair costs due to poorer prediction of failure. 